<h1>TF-IDF</h1>
Author: Tristan

You can learn about TF-IDF here:
https://en.wikipedia.org/wiki/Tf–idf
In brief, it is a way to choose the weight of different terms in a corpus.

It is *not* a way to reduce the number of terms.
Terms can be reduced afterwards using Semantic Analysis (for which there are multiple methods)
https://en.wikipedia.org/wiki/Semantic_analysis_(machine_learning)

In [5]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import *
import scipy

In [6]:
import nlp

In [3]:
data = nlp.proc_text()
data.head()

,index,title,text,links,categories,process
0,3,Art,[[File:Chemin montant dans les hautes herbes -...,"[renoir, senses, bowl, creativity, drawing, pa...","[Art, Non-verbal communication, Basic English ...",art activ creation peopl import attract human ...
1,19,Abbreviation,An '''abbreviation''' is a shorter way to writ...,"[english language, apostrophe, period (punctua...",[Linguistics],abbrevi shorter way write word phrase peopl us...
2,29,Astronomy,[[File:Atlas Coelestis-1.jpg|thumb|280px|18th ...,"[natural science, atmosphere of earth, astrono...",[Astronomy],astronomi natur scienc studi everyth outsid at...
3,50,Browser,{{Distinguish|Web browser}}\n[[Image:Giraffe f...,"[herbivorous, mammal, leaves, shrub, grass, gr...","[Ecology, Zoology]",browser anim usual herbivor mammal eat leav sh...
4,63,Bubonic plague,{{Infobox disease\n| Name = Bubonic...,"[lymphatic system, plague, bacterium, yersinia...","[Plague, Pulmonology]",bubon plagu best known form diseas plagu caus ...


In [7]:
vectorizer = TfidfVectorizer(analyzer = "word", max_df=1.0, min_df=1)
#max_df is the fraction of documents that must have a word before it is ignored.
#min_df is the number of documents that must have a word for it to be considered.
#norm="l2" normalizes each document vector to a (pythagorean) length of 1.
clean_text = data["process"]
weighted_words = vectorizer.fit_transform(clean_text)

In [8]:
#also get the bag of words without weighting for comparison
vectorizer2 = CountVectorizer(analyzer = "word")
unweighted_words = vectorizer2.fit_transform(clean_text)

In [9]:
#Want a function to list words alongside their frequency
def get_word_frequency(sparse_matrix,doc,word_list):
    #find number of distinct words in given document
    num_words = sparse_matrix[doc,:].getnnz()
    #initialize DataFrame
    word_frequency = pd.DataFrame(index=range(num_words), columns=['word','frequency'])
    #convert to another kind of sparse matrix
    cx = scipy.sparse.coo_matrix(sparse_matrix[doc,:])
    #Loop over nonzero elements in the sparse matrix
    #with i = column number, j = weight, and k being the appropriate row of the DataFrame
    for i,j,k in zip(cx.col,cx.data,range(num_words)):
        word_frequency['word'][k] = word_list[i]
        word_frequency['frequency'][k] = j
        
    #Finally, sort the DataFrame
    word_frequency.sort_values('frequency',inplace=True,ascending=False)
    return word_frequency

In [10]:
doc_number = 2
test_freq = get_word_frequency(unweighted_words,doc_number,vectorizer2.get_feature_names())
test_weight = get_word_frequency(weighted_words,doc_number,vectorizer.get_feature_names())
print(str(doc_number) + "th document, before TF-IDF:\n",test_freq[0:10])
print("After TF-IDF:\n",test_weight[0:10])

2th document, before TF-IDF:
           word frequency
99   astronomi        34
108       star        32
153   telescop        27
4          use        18
105      earth        18
110     planet        16
109     galaxi        13
151     observ        13
271       wave        12
101      studi        12
After TF-IDF:
           word frequency
400  astronomi  0.456096
346   telescop    0.3636
391       star  0.257103
389     planet   0.18225
390     galaxi   0.17796
394      earth  0.172158
238   spectrum  0.141482
362   astronom  0.140686
228       wave  0.140149
348     observ  0.136335


In [11]:
data.to_pickle("processed_10k_articles.pkl")

In [14]:
numpy.save("document_term_matrix",weighted_words,allo)

In [17]:
pd.DataFrame(vectorizer.get_feature_names()).to_pickle("term_list.pkl")